In [ ]:
! pip install pytube
! pip install moviepy
! pip install pydub
! pip install pysrt
! pip install -U openai-whisper
! pip install openai

# store the environment variable OPENAI_API_KEY
import os
os.environ['OPENAI_API_KEY'] = '' # PUT YOUR API KEY

# Download the video

In [5]:
# download youtube video

from pytube import YouTube
import os

download_path = "/Users/ahmedibrahim/Downloads"

url = input("Enter the url of the video: ")
yt = YouTube(url)
print("Title: ", yt.title)

# download video on 360p
video = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
print("Video downloaded successfully")
video.download(download_path)

video_path = os.path.join(download_path, video.default_filename)

Title:  حصرياً فيلم العالمي | بطولة يوسف الشريف
Video downloaded successfully


# Load the video

In [6]:
# get the audio of the video
video_name = video_path.split('/')[-1].split('.')[0]
audio_name = f'{video_name}.mp3'
import moviepy.editor as mp
clip = mp.VideoFileClip(video_path)
clip.audio.write_audiofile(audio_name)
os.remove(video_path)

MoviePy - Writing audio in حصرياً فيلم العالمي  بطولة يوسف الشريف.mp3


MoviePy - Done.


# load the audio and reduce noise

In [7]:
# import noisereduce as nr
# import librosa

# y, sr = librosa.load(audio_name, sr=None)
# reduced_noise = nr.reduce_noise(y=y, sr=sr)


In [8]:
# import soundfile as sf

# sf.write(audio_name, reduced_noise, sr)

# Transcribe the audio using whisper

In [9]:
# split the audio into chunks
import pydub
import whisper

model = whisper.load_model("large")
audio = pydub.AudioSegment.from_mp3(audio_name)
transcriptions = []


# split the audio every 30 sec
chunk_length = 60*5
chunk_length_ms = chunk_length * 1000
chunks = []
for i in range(0, len(audio), chunk_length_ms):
    chunks.append(audio[i:i+chunk_length_ms])

for chunk in chunks:
    chunk_name = f'audio_chunks/{video_name}_{i}.mp3'
    chunk.export(chunk_name, format="mp3")
    transcription = model.transcribe(chunk_name, 
                                    language="ar", 
                                    verbose=True,
                                    initial_prompt=transcriptions[-1]['text'] if len(transcriptions) > 0 else None
                                    )
    transcriptions.append(transcription)
    os.remove(chunk_name)
    print(len(transcriptions) * chunk_length / 60, 'minutes done')
os.remove(audio_name)

[00:00.000 --> 00:29.980]  المترجم للقناة
[00:30.000 --> 00:59.980]  ترجمة نانسي قنقر
[01:00.000 --> 01:29.980]  ترجمة نانسي قنقر
[01:30.000 --> 01:59.980]  ترجمة نانسي قنقر
[02:00.000 --> 02:29.980]  ترجمة نانسي قنقر
[02:30.000 --> 02:59.980]  ترجمة نانسي قنقر
[03:00.000 --> 03:29.980]  ترجمة نانسي قنقر
[03:30.000 --> 03:59.980]  ترجمة نانسي قنقر
[04:00.000 --> 04:29.980]  ترجمة نانسي قنقر
[04:30.000 --> 04:59.980]  ترجمة نانسي قنقر
5.0 minutes done
[00:00.000 --> 00:05.860]  ياللي ب perry Maliň 추천ل مهلك الله
[00:22.580 --> 00:24.960]  ما بنمر كامله فنقتش
[00:24.960 --> 00:26.960]  حاضر حاضر
[00:27.960 --> 00:29.960]  ملك نزلت تجيب الطماطم
[00:30.960 --> 00:33.960]  ونعم مفيش طماطم مشترتهاش من الصبح ليه
[00:33.960 --> 00:35.960]  حليني بقى عمال بنتي ترجع
[00:42.960 --> 00:44.960]  تأخرت ليه مقصورة الرقبة
[00:44.960 --> 00:46.960]  أم محمد عايفة رحت أجي من أول الشارع
[00:46.960 --> 00:48.960]  من أول الشارع هات تستجيب
[00:48.960 --> 00:50.960]  يلا ده خلينا الحاجات دي بسرعة لأمك
[00:50

# Forming SRT File from the transcription

In [10]:
# form srt file from the transcriptions
import pysrt
subs = pysrt.SubRipFile()

for i, transcription in enumerate(transcriptions):
    for j, segment in enumerate(transcription['segments']):
        base_seconds = i * chunk_length
        start = segment['start'] + base_seconds
        end = segment['end'] + base_seconds
        text = segment['text']
        subs.append(pysrt.SubRipItem(
            index=j,
            start=pysrt.SubRipTime(seconds=start),
            end=pysrt.SubRipTime(seconds=end),
            text=text
        ))

subs.save(f'arabic_subs/{video_name}.srt', encoding='utf-8')
print("Subtitles saved successfully")


# Translate SRT file using OpenAI

In [11]:
from open_ai_utils import get_openai_response
# load the srt file and translate it

subs = pysrt.open(f'{video_name}.srt', encoding='utf-8')

for sub in subs:
    text = sub.text
    print(text)
    if 'ترجم' in text or 'موسيق' in text or 'قناة' in text or 'إعلان' in text:
        sub.text = ''
        continue
    instructions = "Arabic: " + text + "\nEnglish:"
    response = get_openai_response(instructions)
    sub.text = response
    print(sub.text)
    print('------------------')
subs.save(f'translated_subs/{video_name}_translated.srt', encoding='utf-8')

 المترجم للقناة
 ترجمة نانسي قنقر
 ترجمة نانسي قنقر
 ترجمة نانسي قنقر
 ترجمة نانسي قنقر
 ترجمة نانسي قنقر
 ترجمة نانسي قنقر
 ترجمة نانسي قنقر
 ترجمة نانسي قنقر
 ترجمة نانسي قنقر
 ياللي ب perry Maliň 추천ل مهلك الله


Hey, Perry. Recommend something for me, God help you.
------------------
 ما بنمر كامله فنقتش


We didn't pass the full inspection, so we failed.
------------------
 حاضر حاضر


Okay, okay.
------------------
 ملك نزلت تجيب الطماطم


Malak, go and bring the tomatoes.
------------------
 ونعم مفيش طماطم مشترتهاش من الصبح ليه


Yes, and there are no tomatoes. Why didn't you buy them since morning?
------------------
 حليني بقى عمال بنتي ترجع


Please be patient, my daughter is coming back.
------------------
 تأخرت ليه مقصورة الرقبة


Why are you late, [referring to someone] the neck brace?
------------------
 أم محمد عايفة رحت أجي من أول الشارع


Um Mohamed, I'm exhausted. I've been walking from the beginning of the street.
------------------
 من أول الشارع هات تستجيب


Bring

Traceback (most recent call last):
  File "/Users/ahmedibrahim/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/xh/z5gsc60d3_g6xpyqpzc139f40000gn/T/ipykernel_63264/4182478529.py", line 13, in <module>
    response = get_openai_response(instructions)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ahmedibrahim/Work/Language Projects/Sub_Gen/open_ai_utils.py", line 44, in get_openai_response
  File "/opt/homebrew/lib/python3.11/site-packages/openai/resources/beta/threads/runs/runs.py", line 132, in retrieve
    return self._get(
           ^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/openai/_base_client.py", line 998, in get
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebre

In [12]:
subs.save(f'{video_name}_translated.srt', encoding='utf-8')